# RAG 기본 구조 이해하기

![alt text](<images/1_4_rag process.png>)

## 실습에 활용한 문서
- 국토교통부 25년 주요 업무 추진 계획

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## RAG 기본 파이프라인(1~8단계)


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

아래는 기본적인 RAG 구조 이해를 위한 뼈대코드(skeleton code) 입니다.

각 단계별 모듈의 내용을 앞으로 상황에 맞게 변경하면서 문서에 적합한 구조를 찾아갈 수 있습니다.

(각 단계별로 다양한 옵션을 설정하거나 새로운 기법을 적용할 수 있습니다.)

In [4]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/250114_국토교통부자료.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 18


페이지의 내용을 출력합니다.

In [5]:
print(docs[10].page_content)

- 11 -
ㅇ 
 
상가로 제한된 도시재생씨앗융자* 지원대상을 주상복합 건축물주택 연면적
(
 
이내
50% 
)까지 확대(’25.1)하고, 소상공인을 위해 도로점용료 감면을 
년
’26
까지 연장한다.
쇠퇴도심에 상가창업시설을 조성할 경우 저리 대출 지원주택도시기금
    * 
, 
(
)
4. 편리하고 수준 높은 교통 서비스를 제공하겠습니다.
□지역 간 교통망을 확충하고이동 편의를 제고하기 위한 
, 
교통 서비스도 확대
한다또한
. 
, 교통약자와 물류 현장 종사자등을 위한 지원도 강화한다
 
.
➊전국을 촘촘하게 연결하는 교통망을 구축한다.
□철도망은 경부호남 고속선축에 위치한 대도시권 외에도 전국 주요 거점을 
·
빠르게 연결할 수 있는 “
고속철도망
4 × 4 
구축을 가속화한다
” 
. 경전
선축 보성
임성리
∼
개통
 
(`25.
)
下, 중부내륙선축 수서
광주 및 김천거제 
~
∼
착공(`25.
)
下
등
 
 확정된 고속철도망 사업을신속히 추진하여
 
, 전국 
시간
2
 
생활권을 실현하고 전 국민의 90%에게 고속철도 서비스를 제공할 계획이다.
    * 종축서해전라선중부내륙선중앙선동해선횡축서울속초선경강선대구광주선경전선
(
) 
, 
, 
, 
, (
) 
, 
, 
, 
□간선 도로망도 지속 확충한다. 포항
영덕 
∼
및 새만금
전주 
∼
등고속도로
 
 
개
2
(86.0
)
㎞와 충청내륙및 
 
태백미로
-
등 
 
국도 
개
18
(145.6
)
㎞를 연내 개통
하고, 부산신항
김해
∼
및 
 
계양
강화
∼
등 고속도로 
개
 
2
(42.7
)
㎞와 남양주
춘천
∼
및 
 
고창 흥덕부안 행안 
-
등 국도 
개
16
(145.2
)
㎞를 착공하여 차질
없이 사업을 추진할 계획이다.
□한편
가덕도신공항 착공
, 
(`25.
)
下
대구경북통합신공항 설계 착수
, 
(`25.
)
上 
등 기존 신공항 사업들도 차질없이 추진한다.
전문가 자문을 거쳐 신공항 사업별 기본계획설계를 검토하는 등 안전성 확보
   * 
·
➋누구나 편리

`metadata` 를 확인합니다.

In [6]:
docs[10].__dict__

{'id': None,
 'metadata': {'producer': 'Microsoft: Print To PDF',
  'creator': '',
  'creationdate': '2025-01-10T15:57:58+09:00',
  'source': 'data/250114_국토교통부자료.pdf',
  'file_path': 'data/250114_국토교통부자료.pdf',
  'total_pages': 18,
  'format': 'PDF 1.7',
  'title': '250114(p\x04)__müX_¸H\\_|Áü_ôä_Ÿ@_´|_m€Pµ•',
  'author': '',
  'subject': '',
  'keywords': '',
  'moddate': '2025-01-10T15:57:58+09:00',
  'trapped': '',
  'modDate': "D:20250110155758+09'00'",
  'creationDate': "D:20250110155758+09'00'",
  'page': 10},
 'page_content': '- 11 -\nㅇ \n \n상가로 제한된 도시재생씨앗융자* 지원대상을 주상복합 건축물주택 연면적\n(\n \n이내\n50% \n)까지 확대(’25.1)하고, 소상공인을 위해 도로점용료 감면을 \n년\n’26\n까지 연장한다.\n쇠퇴도심에 상가창업시설을 조성할 경우 저리 대출 지원주택도시기금\n    * \n, \n(\n)\n4. 편리하고 수준 높은 교통 서비스를 제공하겠습니다.\n□지역 간 교통망을 확충하고이동 편의를 제고하기 위한 \n, \n교통 서비스도 확대\n한다또한\n. \n, 교통약자와 물류 현장 종사자등을 위한 지원도 강화한다\n \n.\n➊전국을 촘촘하게 연결하는 교통망을 구축한다.\n□철도망은 경부호남 고속선축에 위치한 대도시권 외에도 전국 주요 거점을 \n·\n빠르게 연결할 수 있는 “\n고속철도망\n4 × 4 \n구축을 가속화한다\n” \n. 경전\n선축 보성\n임성리\n∼\n개통\n \n(`2

In [7]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 47


In [8]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [9]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [10]:
for doc in vectorstore.similarity_search("자율주행차"):
    print(doc.page_content)

산업을 육성
新
하는 한편, 기존 산업의 체질도 적극 개선한다. 
➊미래를 선도해 나갈 
산업을 육성한다. 
新
□자율주행 단계 상용화를 위해 다각적인 실증을 추진한다서울 상암에서
4
. 
 
국내 최초로 완전 무인 자율주행 시범운행을 개시(’25.
)
下한다그간 무인
. 
 
자율주행차 실증은 운전자가 운전석에 탑승한 상태로 진행되었으나이번
, 
에는 탑승자 없이 
이내
50
/h 
㎞
의 속도로 상암 자율주행 시범운행지구 
순환구간(3.2
)
㎞을 달린다. 
ㅇ 
 
최고 100
/h
㎞
의 고속 자율주행과 장거리 자율주행 실증 지원하기 위해 
전국의 고속도로를 자율차 시범운행지구로 지정한다고속도로 시범운행
. 
지구는 자율주행이 용이하지 않은 
구간
IC 
등 일부를 제외
 
하는 네거티브 
방식으로 지정하고, 주요 물류 거점을 오가는 화물차들을 통해 자율주행 
실증을 본격화한다.
ㅇ 한편
 
, 농어촌이나 새벽심야시간 
·
등 대중교통이 부족해 이동에 불편을
국토교통 산업 혁신으로 더 나은 미래를 만들겠습니다
5. 
.
ㅇ
 
전국 고속도로자율차 시범운행지구 지정 및 
 
무인 자율차최초 운행
 
ㅇ
 
디지털 트윈 기반 공장 인
허가 사전진단
·
, MRO 산업 지원방안 마련
 
ㅇ
 
K-City, 
철도
K-
수출
지역 맞춤형 전략으로 
년 
 
, 
25
`
억불
500
달성
 
보도자료
- 14 -
자율주행 시범운행지구 서비스 현황 
< 
>
서울
심야 택시
(
) 
서울
새벽
심야버스
(
) 
·
하동
벽지노선 버스
(
) 
순천
국가정원 셔틀
(
) 
□드론산업의 성장 촉진을 위해 기체의 핵심부품인 모터배터리 등을 국내
·
에서 안정적으로 수급할 수 있도록 ‘드론 제조 생태계 조성방안을 마련
’
(’25.9)하고기술 실증 등을 위한 
, 
드론 특별 자유화구역도 대폭 확대
곳
(47
곳
70
)
→ 
한다.
□도심항공교통
은 글로벌 기술 경쟁력 제고를 위해 연내 아라뱃길 
(UAM)
등 수도권 도심 실증을 본격화하고민간의 사업 참여 유인을 

In [11]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.

In [12]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("삼성전자가 자체 개발한 AI 의 이름은?")

[Document(id='9606c09b-6328-475d-95ae-7bc02dba069b', metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2025-01-10T15:57:58+09:00', 'source': 'data/250114_국토교통부자료.pdf', 'file_path': 'data/250114_국토교통부자료.pdf', 'total_pages': 18, 'format': 'PDF 1.7', 'title': '250114(p\x04)__müX_¸H\\_|Áü_ôä_Ÿ@_´|_m€Pµ•', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-01-10T15:57:58+09:00', 'trapped': '', 'modDate': "D:20250110155758+09'00'", 'creationDate': "D:20250110155758+09'00'", 'page': 13}, page_content='- 14 -\n자율주행 시범운행지구 서비스 현황 \n< \n>\n서울\n심야 택시\n(\n) \n서울\n새벽\n심야버스\n(\n) \n·\n하동\n벽지노선 버스\n(\n) \n순천\n국가정원 셔틀\n(\n) \n□드론산업의 성장 촉진을 위해 기체의 핵심부품인 모터배터리 등을 국내\n·\n에서 안정적으로 수급할 수 있도록 ‘드론 제조 생태계 조성방안을 마련\n’\n(’25.9)하고기술 실증 등을 위한 \n, \n드론 특별 자유화구역도 대폭 확대\n곳\n(47\n곳\n70\n)\n→ \n한다.\n□도심항공교통\n은 글로벌 기술 경쟁력 제고를 위해 연내 아라뱃길 \n(UAM)\n등 수도권 도심 실증을 본격화하고민간의 사업 참여 유인을 높이기 위해\n, \n \n교통관광응급의료 등 \n·\n·\n상용화 모델을 다각화하는 한편\n개 지자체를 대상\n, 2\n으로 시범사업준비를 지원한다\n \n.(’25.\n)\n下  

In [13]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Answer in Korean.

    #Context: 
    {context}

    #Question:
    {question}

    #Answer:"""
)

In [14]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [15]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

생성된 체인에 쿼리(질문)을 입력하고 실행합니다.

In [16]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "25년도 국토교통부 핵심 추진과제는?"
response = chain.invoke(question)
print(response)

2025년도 국토교통부의 핵심 추진과제는 다음과 같습니다:

1. 안전한 대한민국을 만들기 위한 항공 안전 혁신 및 건설 단계별 안전 강화.
2. 서민 주거안정을 위한 공공주택 신축 및 공급.
3. 경쟁력 있는 지방시대 구축.
4. 편리하고 수준 높은 교통서비스 제공.
5. 국토교통산업 혁신.


## 전체 코드

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/250114_국토교통부자료.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "25년도 국토교통부 핵심 추진과제는?"
response = chain.invoke(question)
print(response)

2025년도 국토교통부의 핵심 추진과제는 다음과 같습니다:

1. 안전한 대한민국을 만들겠습니다.
2. 서민 주거안정에 모든 힘을 쏟겠습니다.
3. 경쟁력 있는 지방시대를 열겠습니다.
4. 편리하고 수준 높은 교통 서비스를 제공하겠습니다.
5. 국토교통 산업 혁신으로 더 나은 미래를 만들겠습니다.
